In [ ]:
## Pre-requirement:
## ADFR should be installed properly, and the following scripts were called correspondingly: prepare_ligand, adfr.
## MGLTools should be installed properly.

import re
import os, sys
from os import walk
from re import search
import subprocess
import pandas as pd
import time
import math
import numpy as np

def readSDF (workpath, filename):
    os.chdir(workpath)
    xyzdf=pd.DataFrame()
    pdbread=open(filename,'r', encoding="utf-8"  )
    cordi=0
    for i, pdbline in enumerate (pdbread.readlines()):
        cordxyz=re.split('\s+',pdbline)
        if len(cordxyz)<6:
            continue
        else:
            ATOM_type=re.sub('\d+','',cordxyz[4])
        if ATOM_type.isalpha():    #change to clean
            if pd.isna(ATOM_type):
                continue
            xyzdf.at[cordi,'ATOM']=ATOM_type
            xyzdf.at[cordi,'cordi']=cordi
            xyzdf.at[cordi,'X']=float(cordxyz[1])
            xyzdf.at[cordi,'Y']=float(cordxyz[2])
            xyzdf.at[cordi,'Z']=float(cordxyz[3])
            cordi=cordi+1

    #######
    S_codi=int(xyzdf[xyzdf['ATOM'] == 'S'].index[-1]) 
    if xyzdf.at[S_codi+1,'ATOM']=='C':
        C_codi=S_codi+1
    else:
        C_codi=S_codi-1
        print ('Check ATOM')

    #######
    pdbfileclean=re.sub('.sdf','',filename)+'_cleanH.pdb'
    pdbfile=''
    #atomnum=1
    for i,ATOMcheck in enumerate (xyzdf['ATOM'],start=0):
        x_crd=xyzdf.at[i,'X']
        y_crd=xyzdf.at[i,'Y']
        z_crd=xyzdf.at[i,'Z']
        atomnum=i+1  #atomnum=atomnum+1
        atomname=ATOMcheck
        ligname='LIG'
        chain='A'
        resituenum='777'
        prob=1
        temp=0
        atomtype=ATOMcheck

        if ATOMcheck=='H':
            dXYZ_C_codi=np.sqrt(np.square(xyzdf.at[i,'X']-xyzdf.at[C_codi,'X'])+np.square(xyzdf.at[i,'Y']-xyzdf.at[C_codi,'Y'])+np.square(xyzdf.at[i,'Z']-xyzdf.at[C_codi,'Z']))
            dXYZ_S_codi=np.sqrt(np.square(xyzdf.at[i,'X']-xyzdf.at[S_codi,'X'])+np.square(xyzdf.at[i,'Y']-xyzdf.at[S_codi,'Y'])+np.square(xyzdf.at[i,'Z']-xyzdf.at[S_codi,'Z']))
            if (dXYZ_C_codi<1.2) or (dXYZ_S_codi<1.2) :
                continue

        pdbfile=pdbfile+'HETATM {:>4} {:<4}{:>4}{:>2} {:>3}    {:>8.3f}{:>8.3f}{:>8.3f}  {:>4.2f}  {:>4.2f}          {:>2}'.format(atomnum,atomname,ligname,chain,resituenum,x_crd,y_crd,z_crd,prob,temp,atomtype) + '\n'

    file_w=open(pdbfileclean,'w')
    file_w.write(pdbfile)
    file_w.close()

    C_codi=C_codi+1
    S_codi=S_codi+1
    return (C_codi,S_codi,pdbfileclean)

def CleanCysH (workpath,LIGwithSideChainOUT):
    os.chdir(workpath)
    pdbread=open(LIGwithSideChainOUT,'r')
    LIGwithSideChainOUT_cleanH=re.sub('.pdb','',LIGwithSideChainOUT)+'_cleanH.pdb'
    xyzdf=pd.DataFrame()
    CYSlabel=0
    connectdiscp=''
    cordi=0
    CYSnumber=0
    for i,pdblineread in enumerate (pdbread.readlines()):
        pdblineread=re.sub('\n+','',pdblineread)
        pdbline=re.split('\s+',pdblineread)
        atomhetatm=pdbline[0]
        try:
            cordxyz=[pdbline[6],pdbline[7],pdbline[8]]
            atomnum=pdbline[1]
            atomname=pdbline[2]
            ligname=pdbline[3] #CYS
            chain=pdbline[4]   #A
            resituenum=pdbline[5]
            prob=1
            temp=0
            atomtype=pdbline[11]
        except:
            if atomhetatm == 'COMPND' or atomhetatm =='AUTHOR' or atomhetatm =='REMARK' or atomhetatm =='MASTER':
                continue
            else:
                connectdiscp=connectdiscp+pdblineread+' \n'
                continue
        if atomhetatm=='HETATM' or atomhetatm=='ATOM':
            if atomname=='N':
                CYSlabel=1
            if CYSlabel==1 and atomname!='H':
                atomhetatm='ATOM'
                ligname='CYS'
                CYSnumber=CYSnumber+1
            else:
                atomhetatm='HETATM'
                ligname='LIG'
            if CYSlabel==1 and atomname=='NX':
                atomtype='N'

            xyzdf.at[cordi,'atomhetatm']=atomhetatm
            xyzdf.at[cordi,'atomnum']=atomnum
            xyzdf.at[cordi,'atomname']=atomname
            xyzdf.at[cordi,'ligname']=ligname
            xyzdf.at[cordi,'chain']=chain
            xyzdf.at[cordi,'resituenum']=resituenum
            xyzdf.at[cordi,'X']=float(cordxyz[0])
            xyzdf.at[cordi,'Y']=float(cordxyz[1])
            xyzdf.at[cordi,'Z']=float(cordxyz[2])
            xyzdf.at[cordi,'atomtype']=atomtype
            xyzdf.at[i,'CYSlabel']=CYSlabel

            if CYSlabel==1 and CYSnumber >5:
                CYSlabel=0

            cordi=cordi+1
        else:
            connectdiscp=connectdiscp+pdblineread+' \n'
    xyzdf=xyzdf.dropna(axis=0,subset=['X','Y','Z'])
    xyzdf=xyzdf.reset_index(drop=True)
    for i, ligname1 in enumerate (xyzdf['ligname']):
        if ligname1!='CYS':
            continue
        if xyzdf.at[i,'atomtype']=='H':
            continue
        atomtype1=xyzdf.at[i,'atomtype']

        for j, atomtype2 in enumerate (xyzdf['atomtype']):
            if atomtype2=='H' :
                dXYZ_CYS_codi=np.sqrt(np.square(xyzdf.at[i,'X']-xyzdf.at[j,'X'])+np.square(xyzdf.at[i,'Y']-xyzdf.at[j,'Y'])+np.square(xyzdf.at[i,'Z']-xyzdf.at[j,'Z']))
                if (dXYZ_CYS_codi<1.2) :
                    xyzdf.at[j,'atomhetatm']='delATOM'
    pdbfile=''
    atomnum=0
    for i, atomhetatm in enumerate (xyzdf['atomhetatm']):
        if atomhetatm =='delATOM':
            continue
        x_crd=xyzdf.at[i,'X']
        y_crd=xyzdf.at[i,'Y']
        z_crd=xyzdf.at[i,'Z']
        atomnum=xyzdf.at[i,'atomnum']  # atomnum+1 #
        atomname=xyzdf.at[i,'atomname']
        ligname=xyzdf.at[i,'ligname']
        chain=xyzdf.at[i,'chain']
        resituenum=xyzdf.at[i,'resituenum']
        prob=1
        temp=0
        atomtype=xyzdf.at[i,'atomtype']
        pdbfile=pdbfile+'HETATM {:>4} {:<4}{:>4}{:>2} {:>3}    {:>8.3f}{:>8.3f}{:>8.3f}  {:>4.2f}  {:>4.2f}          {:>2}'.format(atomnum,atomname,ligname,chain,resituenum,x_crd,y_crd,z_crd,prob,temp,atomtype) + '\n'

        if ligname=='CYS' and atomname=='C':
            CYS_CO=str(atomnum)
        if ligname=='CYS' and atomname=='O':
            CYS_OC=str(atomnum)
        if ligname=='CYS' and atomname=='N':
            CYS_N=str(atomnum)
        if ligname=='CYS' and atomname=='CA':
            CYS_CA=str(atomnum)
        if ligname=='CYS' and atomname=='CB':
            CYS_CB=str(atomnum)
        if ligname=='CYS' and atomname=='SG':
            CYS_SG=str(atomnum)

    pdbfile=pdbfile+connectdiscp
    file_w=open(LIGwithSideChainOUT_cleanH,'w')
    file_w.write(pdbfile)
    file_w.close()
    return (LIGwithSideChainOUT_cleanH,CYS_N,CYS_CA,CYS_CO,CYS_OC,CYS_CB,CYS_SG)
def checkPDBQT (filename):
    xyzdf=pd.DataFrame()
    pdbqtread=open(filename,'r')
    ROOT=0
    ENDROOT=0
    checkPDBQTOK=0
    for i, pdbline in enumerate (pdbqtread.readlines()):
        if re.search('^ROOT',pdbline):
            ROOT=ROOT+1
        if re.search('^ENDROOT',pdbline):
            ENDROOT=ENDROOT+1
        if (ROOT==1) and (ENDROOT==0):
            if (re.search('^ATOM',pdbline)) or (re.search('^HETATM',pdbline)):
                cordxyz=re.split('\s+',pdbline)
                atomtype=cordxyz[2]
                resname=cordxyz[3]
                if atomtype =='N' and resname=='CYS':
                    checkPDBQTOK=1
                if atomtype =='CA' and resname=='CYS':
                    checkPDBQTOK=1
    return (checkPDBQTOK)

######################################## Main #######################################
starttime=time.time()

os.chdir('./Covalent_Docking')
workpath=os.getcwd()
### STEP-1
### protein receptor ###

receptorPDBQTfull='ADFRformat_hSTINGfull.pdbqt'
receptorPDBQTdCYS='ADFRformat_hSTINGdCYS.pdbqt' # The CYS91 residue preparation was following the ADFR protocol.

#858,862,856 of CYS91 residue preparation 
#ADFRformat_hSTINGfull.pdbqt
#ATOM    856  N   CYS A  91      90.107 101.410  84.877  1.00 50.30    -0.346 N 
#ATOM    857  HN  CYS A  91      90.342 100.458  84.598  1.00  0.00     0.163 HD
#ATOM    858  CA  CYS A  91      90.769 102.529  84.264  1.00 50.30     0.186 C 
#ATOM    859  CB  CYS A  91      92.286 102.312  84.119  1.00 50.30     0.120 C 
#ATOM    860  SG  CYS A  91      93.137 103.718  83.346  1.00 50.30    -0.095 SA
#ATOM    861  C   CYS A  91      90.208 102.718  82.886  1.00 50.30     0.244 C 
#ATOM    862  O   CYS A  91      90.626 102.063  81.932  1.00 50.30    -0.271 OA
###
#ADFRformat_hSTINGdCYS.pdbqt
#ATOM    856  N   CYS A  91      90.107 101.410  84.877  1.00 50.30    -0.346 N 
#ATOM    858  CA  CYS A  91      90.769 102.529  84.264  1.00 50.30     0.186 C 
#ATOM    861  C   CYS A  91      90.208 102.718  82.886  1.00 50.30     0.244 C 
#ATOM    862  O   CYS A  91      90.626 102.063  81.932  1.00 50.30    -0.271 OA

### ligand ###
docking_ligands=['ExcB.sdf']

for i, dockligand in enumerate (docking_ligands):
    C_codi,S_codi,pdbfileclean=readSDF(workpath, dockligand)
    print ('\n\n\n',i, dockligand, ' S: ',S_codi,' C: ',C_codi , '\n\n')
    ### STEP-2
    LIGCVDOCK=re.sub('.sdf','',dockligand)+'_withSideChain'
    LIGwithSideChainOUT= LIGCVDOCK+'.pdb'
    cmd='../DOCKscripts/prepareCovalent.py --ligand ' +pdbfileclean+ ' --ligindices '+ str(C_codi) +','+ str(S_codi) +' --receptor ' + receptorPDBQTfull +' --residue A:CYS91 --outputfile '+ LIGwithSideChainOUT
    print ('\n',cmd,'\n')
    os.system(cmd)
    cmd='rm '+pdbfileclean
    os.system(cmd)
    ### STEP-3
    LIGwithSideChainOUT_cleanH,CYS_N,CYS_CA,CYS_CO,CYS_OC,CYS_CB,CYS_SG=CleanCysH(workpath,LIGwithSideChainOUT)
    LIGPDBQT=LIGCVDOCK+'.pdbqt'
    print (LIGwithSideChainOUT_cleanH,CYS_N,CYS_CA,CYS_CO,CYS_OC,CYS_CB,CYS_SG)
    checkPDBQTOK=0
    for rootindex in range (int(CYS_N),int(CYS_N)+6,1):
        if checkPDBQTOK==0:
            cmd='prepare_ligand -l '+ LIGwithSideChainOUT_cleanH +' -o '+LIGPDBQT+' -R '+str(rootindex)+ ' -B all '  #N_CA_CO CYS_CA+' '
            print ('\n',cmd,'\n')
            os.system(cmd)
            checkPDBQTOK=checkPDBQT (filename=LIGPDBQT)
        else:
            break

    ### STEP-4 ### binding site flexibility for four arginines residues (Arg83, Arg86, Arg94, and Arg95) 
    CVDOCKtrg=LIGCVDOCK+'.trg'
    CYSCACON=[858,862,856] # change according to 
    CYSresi='CYS91'
    cmd='agfr -r {0} -b user 93.137 103.718 83.346 25 25 25 -c {2} {3} -t {4} -x A:CYS91 -f A:ARG83,ARG86,ARG94,ARG95 -o {1} '.format(receptorPDBQTdCYS,LIGCVDOCK,CYSCACON[0],CYSCACON[1],CYSCACON[2])
    print ('\n',cmd,'\n')
    os.system(cmd)

    # cut ATOM    862  O   CYS A  91      90.626 102.063  81.932  1.00 50.30    -0.271 OA
    ### STEP-5 #  -C 1 2 3 protein CA CO N => chemical pose N CA CO number
    try:
        cmd='adfr -l {0} -t {1} --jobName CVdockRUN -C 1 2 4 --nbRuns 100 --maxEvals 500000 -O '.format(LIGPDBQT,CVDOCKtrg)
        print ('\n',cmd,'\n')
        os.system(cmd)
    except:
        print ('\n\n\n Docking failure. \n\n\n')

    try:
        cmd='rm ._* *_withSideChain.pdb *_withSideChain.trg *_cleanH.pdb *_withSideChain_CVdockRUN.dro residues.log'
        os.system(cmd)
        print ('clean all temp files ')
    except:
        print ('keep all temp files ')

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n Total = ', (time.time()-starttime)/3600, ' hrs \n' )
